In [131]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
tf.executing_eagerly()

False

In [111]:
assert tf.__version__ >= "2.0"
import sys
assert sys.version_info >= (3, 5)
%pip install -q -U tensorflow-addons
%pip install -q -U transformers

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installatio

In [112]:
import sklearn
assert sklearn.__version__ >= "0.20"

In [113]:
np.random.seed(42)
tf.random.set_seed(42)
n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))


## read data 

In [114]:
df = pd.read_csv('CommentsApril2017.csv')

/var/folders/rj/6xj9852x0fxfntwp49y4v84c0000gn/T/ipykernel_917/4029672398.py:1: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('CommentsApril2017.csv')


In [115]:
df.head()
#removing html code and some punctuation 
#will need to remove more in a more efficient manner -__-
df['commentBody'] = df['commentBody'].str.replace('<br/><br/>','')
df['commentBody'] = df['commentBody'].str.replace('<br/>','')
df['commentBody'] = df['commentBody'].str.replace('-','')
df['commentBody'] = df['commentBody'].str.replace('!','')
df['commentBody'] = df['commentBody'].str.replace('?','')
df['commentBody'] = df['commentBody'].str.replace('(','')
df['commentBody'] = df['commentBody'].str.replace(')','')


comments = df['commentBody']

#combine into 2D numpy array  
comments = comments.to_numpy()

#split all commments into characters and combine into one array 
#this is so we can generate based on characters and have one continuous stream of text 
#to generate based on words, we can split each comments by space(???) 
chars = []

for i in range(len(comments)): 
    chars.append(str(comments[i]))

#all comments are combined into one string so we can get the characters 
allChars = str(chars)

/var/folders/rj/6xj9852x0fxfntwp49y4v84c0000gn/T/ipykernel_917/926587282.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['commentBody'] = df['commentBody'].str.replace('?','')
/var/folders/rj/6xj9852x0fxfntwp49y4v84c0000gn/T/ipykernel_917/926587282.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['commentBody'] = df['commentBody'].str.replace('(','')
/var/folders/rj/6xj9852x0fxfntwp49y4v84c0000gn/T/ipykernel_917/926587282.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['commentBody'] = df['commentBod

### preprocess the data 


In [116]:
import tensorflow_text as tf_text

In [155]:
from keras.layers import Dense, GRU,TimeDistributed

In [117]:
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts([allChars])

In [118]:
max_id = len(tokenizer.word_index)
x = tokenizer.document_count
print(max_id)
print(x)

360
1


In [119]:
[encoded] = np.array(tokenizer.texts_to_sequences([allChars])) - 1
#print(encoded[0:30])

### split sequential dataset

In [144]:
#total number of characters in dataset 
# if char_level = False, will return total number of words in dataset 
dataset_size = sum([x for _,x in tokenizer.word_counts.items()])
print('dataset size:',dataset_size)

train_size = dataset_size * 90 // 100
print('train size:',train_size)

#return each character one by one from this set
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])


dataset size: 103680709
train size: 93312638


### creating multiple windows from training set 

In [145]:
n_steps = 100 
#input is shifted one character ahead 
window_length = n_steps +1 
'''
-- by default window creates non-overlapping windows, but we want as large of a dataset as possible, so we set shift too 1 
-- window1 = 0-100, window2 = 1-101 
-- we have created a nested dataset, but RNN expects tensors 
'''
dataset = dataset.window(window_length,shift=1,drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:,:-1],windows[:,1:]))

In [146]:
print(dataset)

<MapDataset element_spec=(TensorSpec(shape=(?, ?), dtype=tf.int64, name=None), TensorSpec(shape=(?, ?), dtype=tf.int64, name=None))>


In [147]:
dataset = dataset.map(
    lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth=max_id),Y_batch))



In [148]:
dataset = dataset.prefetch(1)

<PrefetchDataset element_spec=(TensorSpec(shape=(?, ?, 360), dtype=tf.float32, name=None), TensorSpec(shape=(?, ?), dtype=tf.int64, name=None))>


### create the model 

In [161]:
from keras.models import Sequential
from keras import layers
from keras.models import Model

In [171]:
model = Sequential([
    GRU(128,return_sequences=True,input_shape=[None, max_id],
                         dropout=0.2, recurrent_dropout=0.2),
    GRU(128, return_sequences=True,
                         dropout=0.2, recurrent_dropout=0.2),
    TimeDistributed(Dense(max_id, activation="softmax"))
])
model.summary()
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")




Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_15 (GRU)                (None, None, 128)         188160    
                                                                 
 gru_16 (GRU)                (None, None, 128)         99072     
                                                                 
 time_distributed_7 (TimeDis  (None, None, 360)        46440     
 tributed)                                                       
                                                                 
Total params: 333,672
Trainable params: 333,672
Non-trainable params: 0
_________________________________________________________________


In [177]:
history = model.fit(dataset,epochs = 20,steps_per_epoch=50,verbose=1)

KeyboardInterrupt: 

# What Does This Do? 

consider the sentence: "The earth is an awesome place live"
```python
tokenizer.fit_on_texts(sample_text) 
```
returns the sequence [[1,2,3,4,5,6,7]] where 3--> 'is' and 6-->'place'
```python
sequences = tokenizer.texts_to_sequences('The earth is an great place live') 
```
will return the sequence [[1,2,3,4,6,7]] 

We see that the word 'great' is not considered when we fit the tokenizer on our sample_text, the word awesome is. Therefore, fit_on_texts is used to encode our initial data and the texts_to_sequences function will map before-seen words as well as new ones. 
